# Advanced ANN Techniques

In [ ]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np

## Softmax Activation and Categorical Cross Entropy

In [ ]:
def softmax(output_score):
    result = np.zeros_like(output_score)
    normalization = np.sum(np.exp(output_score))
    for i, score in enumerate(output_score):
        result[i] = np.exp(score) / normalization
    return result

In [ ]:
number_of_classes = 10
raw_output_score = np.array([0, 1, 5, 0, 2, 1, 3, 0, 4, 2], dtype=float)
softmax_output = softmax(raw_output_score)
label = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

fig, axes = plt.subplots(3,1, sharex=True, figsize=(9, 8))
axes[0].bar(np.linspace(0, 9, number_of_classes),
            raw_output_score,
            color="#266662")
axes[0].set_title("Raw Output Score")
axes[1].bar(np.linspace(0, 9, number_of_classes),
            softmax_output,
            color="#266662")
axes[1].set_title("Softmax Activation")
axes[1].set_ylabel("Activation")

axes[2].bar(np.linspace(0, 9, number_of_classes),
        label,
        color="#266662")
axes[2].set_title("Label")
axes[2].set_xlabel("Class Indices")
plt.savefig("softmax.png", dpi=200, bbox_inches="tight")
plt.show()

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_width = x_train.shape[1]
x_height = x_train.shape[2]
np.random.seed(2)

x_val = x_train[:999,...]
y_val = y_train[:999,...]

x_train = x_train[1000:,...]
y_train = y_train[1000:,...]

## Data Preperation

In [ ]:
@tf.function
def normalize_img(data, label):
    return tf.cast(data, tf.float32) / 255., label
    
@tf.function
def one_hot_encoding(data, label, n_classes=10):
    one_hot_labels = tf.one_hot(label, depth=n_classes)
    return data, one_hot_labels

dataset_train = tf.data.Dataset.from_tensor_slices((x_train, y_train))
dataset_train = dataset_train.map(normalize_img)
dataset_train = dataset_train.map(one_hot_encoding)
dataset_train = dataset_train.cache()
dataset_train = dataset_train.shuffle(x_train.shape[0])
dataset_train = dataset_train.batch(64)
dataset_train = dataset_train.prefetch(tf.data.AUTOTUNE)

dataset_val = tf.data.Dataset.from_tensor_slices((x_val, y_val))
dataset_val = dataset_val.map(normalize_img)
dataset_val = dataset_val.map(one_hot_encoding)
dataset_val = dataset_val.batch(64)
dataset_val = dataset_val.cache()
dataset_val = dataset_val.prefetch(tf.data.AUTOTUNE)

dataset_test = tf.data.Dataset.from_tensor_slices((x_test, y_test))
dataset_test = dataset_test.map(normalize_img)
dataset_test = dataset_test.map(one_hot_encoding)
dataset_test = dataset_test.batch(64)
dataset_test = dataset_test.cache()
dataset_test = dataset_test.prefetch(tf.data.AUTOTUNE)

## Building the Model

In [ ]:
input_layer = tf.keras.layers.Input(shape=(x_width, x_height), name="input_layer")
flatten = tf.keras.layers.Flatten(name="flatten")(input_layer)

hidden_layer = tf.keras.layers.Dense(units=512,
                                    activation="relu",
                                    name="hidden_layer_0")(flatten)

hidden_layer = tf.keras.layers.Dense(units=256,
                                    activation="relu",
                                    name="hidden_layer_1")(hidden_layer)

hidden_layer = tf.keras.layers.Dense(units=128,
                                    activation="relu",
                                    name="hidden_layer_2")(hidden_layer)
batch_norm = tf.keras.layers.BatchNormalization(name="batch_norm_2")(hidden_layer)

hidden_layer = tf.keras.layers.Dense(units=64,
                                    activation="relu",
                                    kernel_regularizer=keras.regularizers.L2(0.001),
                                    name="hidden_layer_3")(batch_norm)

hidden_layer = tf.keras.layers.Dense(units=32,
                                    activation="relu",
                                    kernel_regularizer=keras.regularizers.L2(0.001),
                                    name="hidden_layer_4")(hidden_layer)

dropout = tf.keras.layers.Dropout(0.2, name="dropout")(hidden_layer)
output_layer = tf.keras.layers.Dense(units=10,
                                    activation="softmax",
                                    name="output_layer")(dropout)

## Compiling the model

In [ ]:
model = tf.keras.Model(inputs=input_layer, outputs=[output_layer])

model.compile(
    optimizer=tf.keras.optimizers.Adam(0.002),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=[tf.keras.metrics.CategoricalAccuracy()])
model.summary()

## Training the Model

After the data pipeline is configured and the model ist constructed and compile, the training can be performed

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                 patience=5)

lr_reduction = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                                    factor=0.5,
                                                    patience=2,
                                                    min_lr=1e-4)
history = model.fit(
    dataset_train,
    validation_data=dataset_val,
    epochs=2,
    callbacks=[early_stopping, lr_reduction]
    )
model.save("./first_model.h5")

## Analyzing the Metrics

Training and validation metrics are important to monitor the model performance during the training process. It can point towards several problems such as overfitting or instability. 

In [ ]:
print("The history contains the metrics' progress along the training:", history.history.keys())
fig, ax1 = plt.subplots()

ax1.plot(history.epoch, history.history["loss"], label="Training Loss", c="#266662", linestyle="--")
ax1.plot(history.epoch, history.history["val_loss"], label="Valdation Loss", c="#ED5654", linestyle="--")
ax2 = ax1.twinx()
ax2.plot(history.epoch, history.history["categorical_accuracy"], c="#266662", label="Training Accuracy")
ax2.plot(history.epoch, history.history["val_categorical_accuracy"], c="#ED5654", label="Validation Accuracy")
ax1.set_xlabel("Epoch")
ax1.set_ylabel("Loss")
ax2.set_ylabel("Accuracy")
fig.legend()
plt.savefig("all.png", dpi=200, bbox_inches="tight")
plt.show()